In [5]:
# LangChain supports many other chat models. Here, we're using Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


# supports many more optional parameters. Hover on your `ChatOllama(...)`
# class to view the latest available supported parameters
model="llama3"
llm = ChatOllama(model=model)
# prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")

# using LangChain Expressive Language chain syntax
# learn more about the LCEL on
# /docs/concepts/#langchain-expression-language-lcel
# chain = prompt | llm | StrOutputParser()

# topic = {"topic": "Space travel"}

# for brevity, response is printed in terminal
# You can use LangServe to deploy your application for
# production
# print(chain.invoke({"topic": "Space travel"}))
# async for chunks in chain.astream(topic):
#     print(chunks, end="", flush=True)

Why did the astronaut break up with his girlfriend before going to Mars?

Because he needed space!

In [77]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, trim_messages
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

model="llama3"
llm = ChatOllama(model=model)
store = {}

trimmer = trim_messages(
    max_tokens=256,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human",
)




# the prompt can take multiple variables
# in the below example language is another variable
# prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
#         ),
#         MessagesPlaceholder(variable_name="messages"),
#     ]
# )


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a fast and helpful word predictor, given a sentence with the last word partially completed you should give rest of the letters of the word that would grammatically complete the word and no other characters",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

#first should be prompt and then the model, if this is interchanged then it will fail langchain version 
# langchain                 0.2.5              pyhd8ed1ab_1    conda-forge
# langchain-community       0.2.5                    pypi_0    pypi
# langchain-core            0.2.9              pyhd8ed1ab_0    conda-forge
# langchain-text-splitters  0.2.1              pyhd8ed1ab_0    conda-forge

chain = ( RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer) | prompt | llm )



def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# with_message_history = RunnableWithMessageHistory(llm, get_session_history)

# since we have multiple variables we need to specify where to get the messages and hence input_messages_key is passed for RunnableWithMessageHistory
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

config = {"configurable": {"session_id": "new"}}

# to invoke when multiple variables are present

# response = with_message_history.invoke(
#     {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
#     config=config,
# )

# without streaming
# response = with_message_history.invoke(
#     {"messages":[HumanMessage(content="Hi! I'm Bob")]},
#     config=config,
# )

# response.content

for r in with_message_history.stream(
    {"messages":[HumanMessage(content="this is a test to ch")]},
    config=config,
):
    print(r.content, end="", flush=True)

Parent run 4049fe39-4bd7-4b45-a6fa-14c7b90d6f96 not found for run 5f158bc3-2671-444a-85d0-afd27ed3c4f6. Treating as a root run.


eck

In [37]:
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="What's my name?")]},
    config=config,
)

response.content

Parent run eb47eefe-6496-4b2b-8457-dc97460929ef not found for run 967ec865-226d-47f7-920f-7792b474e248. Treating as a root run.


'You told me earlier, Bob! Your name is Bob!'

In [38]:
# config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    {"messages":[HumanMessage(content="Yes you are correct")]},
    config=config,
)

response.content

Parent run 878c72a0-f381-4d94-9b5d-150dadd62fff not found for run 2e8b8e75-86d6-4a88-8f5b-207868c581a1. Treating as a root run.


"I'm glad I was able to get it right the first time. If you need any more help or just want to chat, I'm here for you, Bob!"

In [39]:
# config = {"configurable": {"session_id": "new"}}

response = with_message_history.invoke(
    {"messages":[HumanMessage(content="my age is 25 and i like to swim and write code")]},
    config=config,
)

response.content

Parent run 313ed30a-8fee-4c3f-ba50-a37e87a08ad3 not found for run c887982c-51d8-4f2f-82ca-a2e63a76b169. Treating as a root run.


"That's great, Bob! It sounds like you're quite the active person with a passion for swimming. And wow, writing code too? You must be pretty sharp!\n\nWhat kind of coding do you enjoy doing? Are you more into web development, game development, or maybe something else entirely?\n\nAnd by the way, how's your swimming going? Do you have a favorite spot to swim or any upcoming competitions/plans?"

In [40]:
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="i am a full stack engineer")]},
    config=config,
)

response.content

Parent run 3d528c7b-9d66-4989-92ff-7c91ef44185e not found for run 5a0ac372-4314-4f50-8917-2dc5bd6b6352. Treating as a root run.


"Whoa, that's awesome, Bob! Being a full-stack engineer means you're proficient in both front-end and back-end development. That's impressive!\n\nSo, what kind of projects have you worked on recently? Are you more interested in building scalable APIs or crafting user-friendly interfaces?\n\nAnd, since you mentioned swimming earlier, do you find that it helps you clear your mind and relax after a long day of coding?"

In [41]:
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="what is my name")]},
    config=config,
)

response.content

Parent run 436205a7-1b4e-4a85-a375-e3874c20ccae not found for run 5708d663-47aa-4102-950d-c70494aa7d95. Treating as a root run.


'I remember! Your name is Bob, right?'